### Importing Libraries

In [1]:
import numpy as np 
import cv2 as cv

### PreProcessing Image 

In [2]:
def load_data_preprocess(img_address):
    img = cv.imread(img_address)

    h , w = img.shape[:2] # Width and Height of Image 
    
    proc_img = cv.dnn.blobFromImage(img ,size = (416,416) , scalefactor = 1/255 , crop = False ,swapRB = True )
    
    return img , proc_img , w , h

### Load Weights, Configs and Classes

In [3]:
def read_model_and_labels(weights , configs , label_names):
    
    labels = open(label_names).read().strip().split("\n")
        
    net = cv.dnn.readNet(weights , configs)
    
    return labels , net

In [4]:
def inference(pre_processed_image, w , h , net , labels):
    
    net.setInput(pre_processed_image)
#     layers_name = net.getLayerNames()
#     print(layers_name)
    out_layers = ['yolo_82' , 'yolo_94' , 'yolo_106']
    
    prediction = net.forward(out_layers)
    
    return (prediction)  
    

In [5]:
def final_process(predictions , w , h ):
    ID_classes = []
    confidenses = []
    b_boxes = []
        
    for layer in predictions:
        for detectedObj in layer:
            prob_classes = detectedObj[5:] #Classes with Probabilities
            
            max_index = np.argmax(prob_classes) # find index with max Value(Probability) 
            
            conf = prob_classes[max_index] # extract Class confidence
            
            if conf>0.3 : 
                box = detectedObj[0:4] * np.array([w,h,w,h]) # to convert normalized Image to original sizes
                
                (centerX , centerY , width , height ) = box.astype('int')
                print(centerX , centerY , width , height)
                x = int(centerX - width / 2 )
                y = int(centerY - height / 2 )

                ID_classes.append(max_index)
                confidenses.append(float(conf))
                b_boxes.append([x , y , int(width) , int(height)])
                
    return ID_classes , confidenses , b_boxes
    

In [6]:
def show_result(img ,ID_classes , confidenses , b_boxes , labels):
    
    idxs = cv.dnn.NMSBoxes(b_boxes , confidenses , 0.3 , 0.5 )
    
    for i in idxs.flatten():
        (x , y) = b_boxes[i][0] , b_boxes[i][1]
        (w , h) = b_boxes[i][2] , b_boxes[i][3]
        
        cv.rectangle(img , (x , y) , (x + w , y + h) , (0,255,0) , 1 )
        text = "{} : {:.2f}".format(labels[ID_classes[i]] , confidenses[i])
        cv.putText(img, text , (x , y-10) , cv.FONT_HERSHEY_SIMPLEX , 0.5 , (0,255,0) , 2)
        
    cv.imshow('myWin' , img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
img , proc_img , w , h = load_data_preprocess('images/street.jpg')

labels , net = read_model_and_labels('yolo_files/yolov3.weights' , 'yolo_files/yolov3.cfg' ,'yolo_files/coco.names' )

predictions = inference(proc_img , w , h , net , labels)

ID_classes , confidenses , b_boxes = final_process(predictions , w , h )

show_result(img ,ID_classes , confidenses , b_boxes , labels)

142 191 33 41
67 205 52 51
67 204 64 44
65 206 61 58
217 204 36 29
217 203 46 29
67 212 48 47
65 211 65 45
64 209 61 56
357 217 81 46
358 217 84 48
278 227 27 82
278 228 34 101
358 220 81 44
357 220 82 49
278 232 29 84
278 231 37 96
146 117 8 18
169 135 7 13
441 176 34 12
442 176 35 12
26 194 10 26
196 195 20 14
313 196 11 29
141 201 32 24
217 200 35 21
314 199 11 28
216 205 36 24
492 222 15 25
